In [1]:
from google.colab import drive
drive.mount('/content/drive')
!apt-get install unrar
!pip install torch torchvision

Mounted at /content/drive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar is already the newest version (1:6.1.5-1ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import shutil

# Source path (from your starred files)
source_path = '/content/drive/MyDrive/final_project2.rar'

# Destination path in Colab
dest_path = '/content/final_project2.rar'

# Copy the file
shutil.copy(source_path, dest_path)

!mkdir /content/final_project2
!unrar x /content/final_project2.rar /content/final_project2


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/final_project2.rar

Extracting  /content/final_project2/final_project.docx                     0%  OK 
Creating    /content/final_project2/wood_dataset                      OK
Creating    /content/final_project2/wood_dataset/wood                 OK
Creating    /content/final_project2/wood_dataset/wood/ground_truth    OK
Creating    /content/final_project2/wood_dataset/wood/ground_truth/defect  OK
Extracting  /content/final_project2/wood_dataset/wood/ground_truth/defect/100000000_mask.jpg       0%  OK 
Extracting  /content/final_project2/wood_dataset/wood/ground_truth/defect/100000003_mask.jpg       0%  OK 
Extracting  /content/final_project2/wood_dataset/wood/ground_truth/defect/100000006_mask.jpg       0%  OK 
Extracting  /content/final_project2/wood_dataset/wood/ground_truth/defect/100000007_mask.jpg       0%  OK 
Extracting  /content/final

In [3]:
# ReConPatch (Patch-based Autoencoder Ensemble for Anomaly Detection)
# Full Pipeline: Training + Testing + Anomaly Heatmaps (based on your dataset)

import os
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

# === Config ===
PATCH_SIZE = 32
IMAGE_SIZE = 224
STRIDE = 32
BATCH_SIZE = 64
EPOCHS = 20

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === Utils ===
def extract_patches(img_tensor, patch_size=PATCH_SIZE, stride=STRIDE):
    patches = img_tensor.unfold(1, patch_size, stride).unfold(2, patch_size, stride)
    patches = patches.contiguous().view(3, -1, patch_size, patch_size)
    patches = patches.permute(1, 0, 2, 3)
    return patches

def combine_patches(patches, img_size=IMAGE_SIZE, patch_size=PATCH_SIZE, stride=STRIDE):
    num_patches = int(np.sqrt(len(patches)))
    grid = patches.view(num_patches, num_patches)
    heatmap = torch.zeros((img_size, img_size))
    countmap = torch.zeros((img_size, img_size))
    idx = 0
    for y in range(0, img_size - patch_size + 1, stride):
        for x in range(0, img_size - patch_size + 1, stride):
            patch_score = patches[idx]
            heatmap[y:y+patch_size, x:x+patch_size] += patch_score
            countmap[y:y+patch_size, x:x+patch_size] += 1
            idx += 1
    heatmap /= countmap
    return heatmap

# === Dataset ===
class PatchDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.img_paths = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if f.endswith(('jpg', 'png'))]
        self.transform = transform

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img_tensor = self.transform(img)
        patches = extract_patches(img_tensor)
        return patches, img_tensor, img_path

# === Patch Autoencoder ===
class PatchAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 3, 3, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

# === Transforms ===
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# === Paths ===
train_dir = '/content/final_project2/wood_dataset/wood/train/good'
test_good_dir = '/content/final_project2/wood_dataset/wood/test/good'
test_defect_dir = '/content/final_project2/wood_dataset/wood/test/defect'

In [4]:

# === Load Training Data ===
train_dataset = PatchDataset(train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# === Train ===
model = PatchAutoencoder().to(DEVICE)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

model.train()
for epoch in range(EPOCHS):
    total_loss = 0
    for patches, _, _ in tqdm(train_loader):
        patches = patches[0].to(DEVICE)
        optimizer.zero_grad()
        recon = model(patches)
        loss = criterion(recon, patches)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss / len(train_loader):.4f}")

100%|██████████| 70/70 [00:05<00:00, 13.80it/s]


Epoch 1/20, Loss: 0.0640


100%|██████████| 70/70 [00:03<00:00, 19.41it/s]


Epoch 2/20, Loss: 0.0066


100%|██████████| 70/70 [00:03<00:00, 20.98it/s]


...

Epoch 18/20, Loss: 0.0007


100%|██████████| 70/70 [00:04<00:00, 16.20it/s]


Epoch 19/20, Loss: 0.0007


100%|██████████| 70/70 [00:03<00:00, 21.52it/s]

Epoch 20/20, Loss: 0.0007


In [5]:

# Save model
torch.save(model.state_dict(), "patch_autoencoder.pth")

In [6]:

# === Evaluation ===
def evaluate(model, test_dir, save_prefix):
    model.eval()
    dataset = PatchDataset(test_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=1, shuffle=False)
    with torch.no_grad():
        for patches, img_tensor, path in tqdm(loader):
            patches = patches[0].to(DEVICE)
            recon = model(patches)
            error = torch.mean((patches - recon) ** 2, dim=[1, 2, 3])
            heatmap = combine_patches(error.cpu())

            # Normalize heatmap for visualization
            heatmap_np = heatmap.numpy()
            heatmap_np = (heatmap_np - heatmap_np.min()) / (heatmap_np.max() - heatmap_np.min() + 1e-8)

            # Save heatmap overlay
            img_np = img_tensor[0].permute(1, 2, 0).numpy()
            img_np = (img_np * 0.5 + 0.5).clip(0, 1)
            plt.imshow(img_np)
            plt.imshow(heatmap_np, cmap='jet', alpha=0.5)
            plt.axis('off')
            fname = os.path.basename(path[0])
            plt.savefig(f"{save_prefix}_{fname}", bbox_inches='tight', pad_inches=0)
            plt.close()

# Run Evaluation
model.load_state_dict(torch.load("patch_autoencoder.pth"))
evaluate(model, test_good_dir, "heatmap_good")
evaluate(model, test_defect_dir, "heatmap_defect")


100%|██████████| 71/71 [00:07<00:00,  9.05it/s]


In [7]:
evaluate(model, test_good_dir, "heatmap_good")
evaluate(model, test_defect_dir, "heatmap_defect")

100%|██████████| 71/71 [00:09<00:00,  7.48it/s]


In [8]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

def score_dataset(model, test_dirs, labels):
    model.eval()
    scores = []
    with torch.no_grad():
        for dir_path, label in zip(test_dirs, labels):
            dataset = PatchDataset(dir_path, transform=transform)
            loader = DataLoader(dataset, batch_size=1, shuffle=False)
            for patches, _, _ in loader:
                patches = patches[0].to(DEVICE)
                recon = model(patches)
                error = torch.mean((patches - recon) ** 2, dim=[1, 2, 3])  # per-patch
                image_score = error.mean().item()  # mean over patches
                scores.append((image_score, label))
    return scores

# Run scoring
test_dirs = [test_good_dir, test_defect_dir]
labels = [0, 1]
all_scores = score_dataset(model, test_dirs, labels)

# Separate scores and labels
y_scores = [s for s, _ in all_scores]
y_true = [l for _, l in all_scores]

auc = roc_auc_score(y_true, y_scores)
print(f"ROC AUC: {auc:.4f}")


ROC AUC: 0.7636
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
